In [ ]:
import numpy as np
from kalman import Kalman, LDS

In [ ]:
# 1d constant dynamics, no noise
def test_1d():
    """Test that the Kalman filter works on a 1D constant system with no noise"""
    A = np.array([[1]])
    B = np.array([[0]])
    C = np.array([[1]])
    D = np.array([[0]])
test_1d()

In [ ]:
# 1d constant dynamics, measure noise
# 1d constant dynamics, process noise
# 1d constant dynamics, process + measure noise

# 1d constant dyanmics, process + measure noise